In [34]:
import pandas as pd

df = pd.read_csv('schedule.csv')
moorings = pd.read_csv('moorings.csv')

In [35]:
map_col = df.head().loc[0]
map_col

Уникальный ID записи                                                                                 id
ID причала                                                                                      dock.id
Название причала                                                                              dock.name
Название судна - отсутсвует если отменено                                                     ship.name
Название маршрута                                                                       route.nameroute
Швартовочное место                                                                position.berth_letter
Дата начала действия записи расписания в UTC                                        timetable.startdate
Дата окончания действиязаписи  расписания в UTC                                       timetable.enddate
Время подхода судна в минутах от начала суток                                       timetable.starttime
Время отхода судна в минутах от начала суток                    

In [36]:
def correct_data(data: pd.DataFrame) -> pd.DataFrame:
    '''
    Приводит датасет в нужный формат
    '''
    return data.rename(dict(zip(data.columns, data.loc[0].values)), axis=1).drop(0)

In [37]:
df = correct_data(df)
df.head()

,id,dock.id,dock.name,ship.name,route.nameroute,position.berth_letter,timetable.startdate,timetable.enddate,timetable.starttime,timetable.endtime,timetable.duration,changed_schedule.ship_name,changed_schedule.type,Начало расписания,Окончание расписания,Причаливание,Отход
1,1093216,45,Воробьевы горы,Алина Браво,Северный 1А,A,1719954000,1731790800,1,5,4,NaN,NaN,7/3/2024,11/17/2024,12:01:00 AM,12:05:00 AM
2,1094166,32,Нескучный сад,М-250,Исторический,C,1719954000,1731790800,1,5,4,NaN,NaN,7/3/2024,11/17/2024,12:01:00 AM,12:05:00 AM
3,1094538,32,Нескучный сад,Москва-24,Северный,B,1719954000,1731790800,1,4,3,NaN,NaN,7/3/2024,11/17/2024,12:01:00 AM,12:04:00 AM
4,1096121,45,Воробьевы горы,NaN,Северный 1А,A,1719954000,1719954000,1,5,4,Алина Браво,Отмена,7/3/2024,7/3/2024,12:01:00 AM,12:05:00 AM
5,1096896,32,Нескучный сад,NaN,Исторический,C,1719954000,1719954000,1,5,4,М-250,Отмена,7/3/2024,7/3/2024,12:01:00 AM,12:05:00 AM


In [38]:
moorings = correct_data(moorings)
moorings.head()

,Docs.id,Docs.name,Docs.address,Docs.river,Docs.latitude,Docs.longitude
1,31,Парк Горького,Пушкинская набережная,Москва,"37,597184","55,73013"
2,32,Нескучный сад,"г. Москва, Вверх по течению о.Москва от Крымск...",Москва,"37,590694","55,722427"
3,33,Крымский мост,Фрунзенская набережная напротив д.10,Москва,"37,596061","55,732427"
4,39,Марьино,"Батайский проезд, д.33",Москва,"37,725065","55,641785"
5,41,Печатники,"ул.Гурьянова, д.35",Москва,"37,714149","55,683699"


In [39]:
df = df.drop('id', axis=1)

In [40]:
df = df.drop('dock.id', axis=1)
moorings = moorings.drop('Docs.id', axis=1)

Временно удалим атрибуты timetable.startdate, timetable.enddate

In [41]:
df = df.drop(['timetable.startdate', 'timetable.enddate'], axis=1)

In [42]:
df = df.drop(['timetable.starttime', 'timetable.endtime'], axis=1)

In [43]:
df = df.drop(['Начало расписания', 'Окончание расписания'], axis=1)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7887 entries, 1 to 7887
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   dock.name                   7887 non-null   object
 1   ship.name                   4129 non-null   object
 2   route.nameroute             7887 non-null   object
 3   position.berth_letter       7887 non-null   object
 4   timetable.duration          7887 non-null   object
 5   changed_schedule.ship_name  2254 non-null   object
 6   changed_schedule.type       2254 non-null   object
 7   Причаливание                7743 non-null   object
 8   Отход                       7887 non-null   object
dtypes: object(9)
memory usage: 554.7+ KB


In [45]:
col_numeric = ['timetable.duration']

df[col_numeric] = df[col_numeric].astype(int)

In [46]:
df.describe(include='object')

,dock.name,ship.name,route.nameroute,position.berth_letter,changed_schedule.ship_name,changed_schedule.type,Причаливание,Отход
count,7887,4129,7887,7887,2254,2254,7743,7887
unique,36,82,11,14,41,1,956,924
top,Нескучный сад,Чижик-II,Исторический,A,Чижик-II,Отмена,12:59:00 AM,10:00:00 AM
freq,1157,900,3159,2803,1188,2254,114,243


In [47]:
df['Причаливание'] = pd.to_datetime(df['Причаливание'])
df['Отход'] = pd.to_datetime(df['Отход'])

/tmp/ipykernel_257017/832309581.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Причаливание'] = pd.to_datetime(df['Причаливание'])
/tmp/ipykernel_257017/832309581.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Отход'] = pd.to_datetime(df['Отход'])


In [48]:
df.isna().sum()

dock.name                        0
ship.name                     3758
route.nameroute                  0
position.berth_letter            0
timetable.duration               0
changed_schedule.ship_name    5633
changed_schedule.type         5633
Причаливание                   144
Отход                            0
dtype: int64

пропуски в ship.name = отсутсвуют или отменены (кол-во)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7887 entries, 1 to 7887
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   dock.name                   7887 non-null   object        
 1   ship.name                   4129 non-null   object        
 2   route.nameroute             7887 non-null   object        
 3   position.berth_letter       7887 non-null   object        
 4   timetable.duration          7887 non-null   int64         
 5   changed_schedule.ship_name  2254 non-null   object        
 6   changed_schedule.type       2254 non-null   object        
 7   Причаливание                7743 non-null   datetime64[ns]
 8   Отход                       7887 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 554.7+ KB


In [50]:
import numpy as np

df['changed_schedule.type'] = df['changed_schedule.type'].replace('Отмена', np.nan)

/tmp/ipykernel_257017/3812200950.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['changed_schedule.type'] = df['changed_schedule.type'].replace('Отмена', np.nan)


In [51]:
df.isna().sum()

dock.name                        0
ship.name                     3758
route.nameroute                  0
position.berth_letter            0
timetable.duration               0
changed_schedule.ship_name    5633
changed_schedule.type         7887
Причаливание                   144
Отход                            0
dtype: int64

In [52]:
df.shape

(7887, 9)

In [53]:
df = df.drop('changed_schedule.type', axis=1)

In [54]:
df = df.dropna(subset='ship.name')

In [55]:
df = df.drop('changed_schedule.ship_name', axis=1)

In [56]:
df = df.reset_index(drop='first')

dock_start фильтр -> ищем dock_end в route.nameroute 

In [57]:
dock_start = 'Воробьевы горы'
dock_end = 'Красный октябрь'
datetime_cur = pd.to_datetime('2024-12-01 15:30:00')

Название маршрутов, проходящих через данную остановку

In [58]:
nameroute_arr = df[df['dock.name'] == dock_start]['route.nameroute'].unique()
nameroute_arr

array(['Северный 1А', 'Исторический', 'Северный', 'Разовый подход'],
      dtype=object)

Ищем маршруты в которых есть пункт назначения (возможно их нет) и пречаливание которых больше текущего времени

In [59]:
df[(df['route.nameroute'] == 'Исторический') & (df['dock.name'] == dock_end) & (df['Причаливание'] >= datetime_cur)]

,dock.name,ship.name,route.nameroute,position.berth_letter,timetable.duration,Причаливание,Отход
1845,Красный октябрь,Москва-51,Исторический,A,4,2024-12-01 15:34:00,2024-12-01 15:38:00
1884,Красный октябрь,Адмиралъ,Исторический,A,4,2024-12-01 15:49:00,2024-12-01 15:53:00
1931,Красный октябрь,Москва-61,Исторический,A,4,2024-12-01 16:04:00,2024-12-01 16:08:00
2013,Красный октябрь,Полковник Новгородов,Исторический,A,4,2024-12-01 16:19:00,2024-12-01 16:23:00
2061,Красный октябрь,Москва-28,Исторический,A,4,2024-12-01 16:34:00,2024-12-01 16:38:00
...,...,...,...,...,...,...,...
3771,Красный октябрь,Алина Танго,Исторический,A,4,2024-12-01 22:49:00,2024-12-01 22:53:00
3843,Красный октябрь,Москва-51,Исторический,A,4,2024-12-01 23:04:00,2024-12-01 23:08:00
3868,Красный октябрь,Адмиралъ,Исторический,A,4,2024-12-01 23:19:00,2024-12-01 23:23:00
3921,Красный октябрь,Москва-61,Исторический,A,4,2024-12-01 23:34:00,2024-12-01 23:38:00


Ищем среди ship.name те, которые еще не отплыли относительно текущего времени

In [60]:
df[(df['ship.name'] == 'Полковник Новгородов') & (df['dock.name'] == dock_start) & (df['Отход'] >= datetime_cur)]

,dock.name,ship.name,route.nameroute,position.berth_letter,timetable.duration,Причаливание,Отход
1840,Воробьевы горы,Полковник Новгородов,Исторический,B,10,2024-12-01 15:32:00,2024-12-01 15:42:00
2031,Воробьевы горы,Полковник Новгородов,Разовый подход,B,5,2024-12-01 16:25:00,2024-12-01 16:30:00
2357,Воробьевы горы,Полковник Новгородов,Разовый подход,B,5,2024-12-01 17:40:00,2024-12-01 17:45:00
2456,Воробьевы горы,Полковник Новгородов,Исторический,B,10,2024-12-01 18:02:00,2024-12-01 18:12:00
2658,Воробьевы горы,Полковник Новгородов,Разовый подход,B,5,2024-12-01 18:55:00,2024-12-01 19:00:00
3057,Воробьевы горы,Полковник Новгородов,Разовый подход,B,5,2024-12-01 20:10:00,2024-12-01 20:15:00
3153,Воробьевы горы,Полковник Новгородов,Исторический,B,10,2024-12-01 20:32:00,2024-12-01 20:42:00
3357,Воробьевы горы,Полковник Новгородов,Разовый подход,B,5,2024-12-01 21:25:00,2024-12-01 21:30:00
3841,Воробьевы горы,Полковник Новгородов,Исторический,B,10,2024-12-01 23:02:00,2024-12-01 23:12:00


In [61]:
data = pd.DataFrame(columns=df.columns)
for nameroute in nameroute_arr:
    # пункт B
    ship_list = df[(df['route.nameroute'] == nameroute) & (df['dock.name'] == dock_end) & (df['Причаливание'] > datetime_cur)]['ship.name'].unique()
    for ship_name in ship_list:
        # пункт A
        data = pd.concat([data, df[(df['ship.name'] == ship_name) & (df['dock.name'] == dock_start) & (df['Отход'] >= datetime_cur)]])
data.sort_values('Отход')

/tmp/ipykernel_257017/3605069621.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, df[(df['ship.name'] == ship_name) & (df['dock.name'] == dock_start) & (df['Отход'] >= datetime_cur)]])


,dock.name,ship.name,route.nameroute,position.berth_letter,timetable.duration,Причаливание,Отход
1855,Воробьевы горы,ОМ-349,Разовый подход,B,5,2024-12-01 15:35:00,2024-12-01 15:40:00
1840,Воробьевы горы,Полковник Новгородов,Исторический,B,10,2024-12-01 15:32:00,2024-12-01 15:42:00
1865,Воробьевы горы,М-250,Разовый подход,B,5,2024-12-01 15:40:00,2024-12-01 15:45:00
1882,Воробьевы горы,Москва-28,Исторический,B,10,2024-12-01 15:47:00,2024-12-01 15:57:00
1927,Воробьевы горы,М-250,Исторический,B,10,2024-12-01 16:02:00,2024-12-01 16:12:00
...,...,...,...,...,...,...,...
3966,Воробьевы горы,Чижик-II,Исторический,B,10,2024-12-01 23:47:00,2024-12-01 23:57:00
3967,Воробьевы горы,Чижик-II,Исторический,B,10,2024-12-01 23:47:00,2024-12-01 23:57:00
3938,Воробьевы горы,Олимпиада,Исторический,B,10,2024-12-01 23:47:00,2024-12-01 23:57:00
3935,Воробьевы горы,Чижик-II,Исторический,B,10,2024-12-01 23:47:00,2024-12-01 23:57:00


Примем, что все корабли с одинаковым названием движутся по одному и тому же маршруту одинаковое кол-во времени

In [62]:
moorings.dtypes

Docs.name         object
Docs.address      object
Docs.river        object
Docs.latitude     object
Docs.longitude    object
dtype: object

In [63]:
moorings['Docs.latitude'] = moorings['Docs.latitude'].apply(lambda x: float(x.replace(',', '.')))
moorings['Docs.longitude'] = moorings['Docs.longitude'].apply(lambda x: float(x.replace(',', '.')))

In [64]:
def distance(p1: float, p2: float, q1: float, q2: float) -> float:
    '''
    Вычисляет манхетанское расстояние (в км)
    '''
    k = 0.95 / 0.01419299999999879 # перевод в км (ПГ - Нескучный шаг)
    return (abs(p1 - q1) + abs(p2 - q2)) * k

def cord_to_dock(latitude: float, longitude: float) -> list:
    '''
    Возвращает список причалов, отсортированных по близости к текущей точке
    '''
    dock_names = moorings['Docs.name']
    res_list = []
    
    for dock_name in dock_names:
        dock_data = moorings[moorings['Docs.name'] == dock_name]
        dock_latitide = dock_data['Docs.latitude'].values[0] 
        dock_longitude = dock_data['Docs.longitude'].values[0]
        
        res_list.append((dock_name, distance(latitude, longitude, dock_latitide, dock_longitude)))
    
    return sorted(res_list, key=lambda x: x[1])

def dock_shedule(dock_start, dock_end, datetime_cur):
    '''
    Релевантное расписание для причала
    '''
    data = pd.DataFrame(columns=df.columns)
    nameroute_arr = df[df['dock.name'] == dock_start]['route.nameroute'].unique()
    
    for nameroute in nameroute_arr:
        # пункт B
        ship_list = df[(df['route.nameroute'] == nameroute) & (df['dock.name'] == dock_end) & (df['Причаливание'] > datetime_cur)]['ship.name'].unique()
        for ship_name in ship_list:
            # пункт A
            data = pd.concat([data, df[(df['ship.name'] == ship_name) & (df['dock.name'] == dock_start) & (df['Отход'] >= datetime_cur)]])
    
    return data.sort_values('Отход')
    

def shedule(latitude: float, longitude: float, dock_end: str, datetime_cur: str) -> pd.DataFrame:
    '''
    Создание релевантного расписания.
    
    Аргументы функции:  
    latitude - долгота точки клиента
    longitude - широта точки клиента
    dock_end - пункт назначения, название причала
    datetime_cur - текущая дата и время, в формате 'YYYY-MM-DD HH:MM:SS' (24-часовом формате времени)
    '''
    nearest_docks = cord_to_dock(latitude, longitude)[:3]
    datetime_cur = pd.to_datetime(datetime_cur)
    
    res_df = pd.DataFrame(columns=df.columns)
    for dock in nearest_docks:
        res_df = pd.concat([res_df, dock_shedule(dock[0], dock_end, datetime_cur)])
        
    return res_df.sort_values('Отход')

In [65]:
cord_to_dock(37.597184, 55.73013)

[('Парк Горького', np.float64(0.0)),
 ('Крымский мост', np.float64(0.22891566265051647)),
 ('Нескучный сад', np.float64(0.95)),
 ('Патриарший', np.float64(1.6597054886209537)),
 ('Красный октябрь', np.float64(1.9037483266399415)),
 ('Третьяковский', np.float64(2.3829986613119334)),
 ('Киевский', np.float64(2.602878179384121)),
 ('Андреевский', np.float64(2.906961178045813)),
 ('Зарядье', np.float64(3.4441767068272426)),
 ('Лужники – Северный', np.float64(3.699866131192171)),
 ('Новоспасский', np.float64(3.7723560910306686)),
 ('Автозаводский мост', np.float64(3.7979919678722753)),
 ('Китай-город', np.float64(3.8175368139227874)),
 ('Трёхгорный', np.float64(3.839892904953256)),
 ('ЗИЛ', np.float64(4.142322623829317)),
 ('Лужники - Центральный', np.float64(4.329585006693792)),
 ('Сити – Багратион', np.float64(4.571753681392334)),
 ('Воробьевы горы', np.float64(4.601740294511866)),
 ('Сити – Экспоцентр', np.float64(4.6186746987952425)),
 ('Сити – Центральный', np.float64(4.815930388219551

In [66]:
test_shedule = shedule(37.597184, 55.73013, 'Воробьевы горы', '2024-12-01 15:30:00')
test_shedule

/tmp/ipykernel_257017/2318264412.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, df[(df['ship.name'] == ship_name) & (df['dock.name'] == dock_start) & (df['Отход'] >= datetime_cur)]])
/tmp/ipykernel_257017/2318264412.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_df = pd.concat([res_df, dock_shedule(dock[0], dock_end, datetime_cur)])


,dock.name,ship.name,route.nameroute,position.berth_letter,timetable.duration,Причаливание,Отход
1836,Нескучный сад,Снегири-4,Северный,B,3,2024-12-01 15:31:00,2024-12-01 15:34:00
1834,Нескучный сад,Адмиралъ,Исторический,C,4,2024-12-01 15:31:00,2024-12-01 15:35:00
1844,Нескучный сад,Прага,Северный,A,4,2024-12-01 15:34:00,2024-12-01 15:38:00
1857,Нескучный сад,Москва-21,Северный 1А,B,3,2024-12-01 15:38:00,2024-12-01 15:41:00
1870,Нескучный сад,Михаил Сеспель,Северный 1А,A,4,2024-12-01 15:41:00,2024-12-01 15:45:00
...,...,...,...,...,...,...,...
3927,Нескучный сад,Удача,Северный 1А,B,3,2024-12-01 23:38:00,2024-12-01 23:41:00
3932,Нескучный сад,Снегири-4,Северный,B,3,2024-12-01 23:46:00,2024-12-01 23:49:00
3934,Нескучный сад,Москва-28,Исторический,C,4,2024-12-01 23:46:00,2024-12-01 23:50:00
3975,Нескучный сад,Прага,Северный,A,4,2024-12-01 23:49:00,2024-12-01 23:53:00
